In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import *
import statsmodels.stats.multitest as smm

In [2]:
data = pd.read_csv("data/gene_high_throughput_sequencing.csv")

In [3]:
data

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,STT5750_Breast_021_DCIS,cancer,1.466240,3.689955,21.331981,19.359546,23.505609,11.576176,11.325851,80.572057,...,2.807635,1.466240,2.807635,1.466240,2.807635,3.689955,18.449777,1.466240,1.466240,1.466240
68,STT5751_Breast_021_IDC,cancer,2.492410,12.279444,30.087759,17.703926,26.064890,10.707081,11.520120,65.041865,...,2.492410,2.492410,1.301619,3.275668,1.301619,1.301619,8.601194,1.301619,1.301619,1.301619
69,STT5758_Breast_022_IDC,cancer,3.530477,20.006038,53.924651,25.449565,30.949995,16.794696,17.121366,70.958462,...,1.176826,1.176826,1.176826,2.961613,1.176826,1.176826,10.764365,1.176826,1.176826,1.176826
70,STT5763_Breast_022_IDC,cancer,3.733734,8.860505,32.538666,21.585069,24.987992,11.460224,11.201202,67.767125,...,2.840946,1.483635,1.483635,1.483635,1.483635,1.483635,8.522837,1.483635,1.483635,1.483635


### P1

#### для групп normal (control) и early neoplasia (treatment)

In [4]:
control_1 = data.query("Diagnosis == 'normal'").drop(["Diagnosis", "Patient_id"], axis=1)
treatment_1 = data.query("Diagnosis == 'early neoplasia'").drop(["Diagnosis", "Patient_id"], axis=1)
control_2 = data.query("Diagnosis == 'early neoplasia'").drop(["Diagnosis", "Patient_id"], axis=1)
treatment_2 = data.query("Diagnosis == 'cancer'").drop(["Diagnosis", "Patient_id"], axis=1)

In [5]:
mask1 = sc.stats.ttest_ind(control_1, treatment_1, equal_var=False).pvalue < .05

In [6]:
mask2 = sc.stats.ttest_ind(control_2, treatment_2, equal_var=False).pvalue < .05

In [7]:
mask1.sum()

1575

In [8]:
mask2.sum()

3490

### P2

In [15]:
sign_1 = sc.stats.ttest_ind(control_1, treatment_1, equal_var=False)
sign_2 = sc.stats.ttest_ind(control_2, treatment_2, equal_var=False)

In [54]:
h_s_1 = smm.multipletests(sign_1.pvalue, alpha = 0.025, method = 'fdr_bh')
h_s_2 = smm.multipletests(sign_2.pvalue, alpha = 0.025, method = 'fdr_bh')

In [40]:
def fold_change(T, C):
    arr = []
    for v in zip(T, C):
        arr.append(v[0] / v[1] if v[0] >= v[1] else - v[1] / v[0])
    arr = np.array(arr)
    return np.sum(np.abs(arr) > 1.5)

In [55]:
c1 = control_1.mean()[h_s_1[1] < .025]
t1 = treatment_1.mean()[h_s_1[1] < .025]

In [56]:
fold_change(t1.values, c1.values)

4

In [57]:
c2 = control_2.mean()[h_s_2[1] < .025]
t2 = treatment_2.mean()[h_s_2[1] < .025]

In [58]:
fold_change(t2.values, c2.values)

524